# 단어 빈도수 기반 자연어 처리
> 단어의 빈도수
- 의미가 있는 단어임을 알 수 있음
- 문장 구분, 문맥을 파악할 수 있음.

In [2]:
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from tensorflow.keras.datasets import imdb
#  한국어 형태소 분석기
from konlpy.tag import Twitter
from konlpy.tag import Okt
from konlpy.tag import Kkma 
from konlpy.tag import Twitter
from pprint import pprint
import nltk
from nltk.classify.scikitlearn import SklearnClassifier
from wordcloud import WordCloud, STOPWORDS
from gensim import corpora, models
import numpy  as np
from PIL import Image
from wordcloud import ImageColorGenerator
import glob
import re
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
%matplotlib inline

In [38]:
corpus = ['you know I want our love',
         'I like you',
         'what should I do']

# 1. 전체 단어의 수와 각 단어의 빈도수 조사

## 1.1 단어 빈도수 세기

In [39]:
srt = ''.join(corpus)
words =srt.split()

In [40]:
freq = {}
for w in words:
    freq[w] = freq.get(w, 0)+1

In [41]:
freq

{'you': 1,
 'know': 1,
 'I': 2,
 'want': 1,
 'our': 1,
 'loveI': 1,
 'like': 1,
 'youwhat': 1,
 'should': 1,
 'do': 1}

## 1.2 CountVectorizer: 단어 사전 만들기 + vectorize
- 단어의 개수와 상관없이 고정된 길이의 벡터를 만들어서 
- 단어가 있는지 없는지 보고, 있으면 1, 없으면 0으로 만들기 
- 문장을 이루는 count를 계산해서 onehot encoding으로 바꾸기
- 단어의 순서는 상관없음. 

In [42]:
vector = CountVectorizer()
tf = vector.fit_transform(corpus)

# dense한 행렬
print(tf.toarray())

# sparse한 행렬(희소 행렬)-> 메모리를 적게 사용
print(tf)  # (행, 열)에서 1인 값

[[0 1 0 1 1 0 1 0 1]
 [0 0 1 0 0 0 0 0 1]
 [1 0 0 0 0 1 0 1 0]]
  (0, 8)	1
  (0, 1)	1
  (0, 6)	1
  (0, 4)	1
  (0, 3)	1
  (1, 8)	1
  (1, 2)	1
  (2, 7)	1
  (2, 5)	1
  (2, 0)	1


> 불용어 처리(stop word) 
- 글자수 1개인 것은 다 빼기 때문에 문장 길이와 1인 성분의 개수가 안맞음

> 벡터라이즈
- 유사도를 구하기 위해 거리를 잴 수 있음.
- 분포도 파악할 수 있음.

In [43]:
tf.toarray().shape  

(3, 9)

In [44]:
# 단어 사전:-> 인덱스갑 의미(빈도수 아님)
vector.vocabulary_  

{'you': 8,
 'know': 1,
 'want': 6,
 'our': 4,
 'love': 3,
 'like': 2,
 'what': 7,
 'should': 5,
 'do': 0}

In [45]:
print(vector.vocabulary_['you'])
print(vector.vocabulary_.get('you')) 

8
8


In [46]:
words = vector.get_feature_names()
for word in words: 
    print(word)
    

do
know
like
love
our
should
want
what
you


In [47]:
for key in vector.vocabulary_: 
    print(key, vector.vocabulary_[key])

you 8
know 1
want 6
our 4
love 3
like 2
what 7
should 5
do 0


## 1.3. TF-IDF(Term Frequency - Inverse Document Frequency)

### TF, DF, TF-IDF 개념
> TF : 내 문서만 보는 관점
- 현재 문서(문장)에서 나타난 빈도수 (위의 벡터는 전체 문장에서)
- tf가 높으면 그 문장에서 중요한 단어임.
- 내 문서에서만 많이 나타나고, 다른 문장에서는 안나타날 수도 있음.
- tf만 보기에는 문맥에 따라 단어의 중요도가 다를 수 있음.
- <font color = 'red'>한문장에서 빈도수 / 문장 전체 단어 수</font>

> DF(Documents Frequency): 다른 문서도 같이 보는 관점
- 단어가 나타난 문서의 수
- df가 높으면 흔한 단어이다. (the, a,...)
- 얼마나 이 단어가 전체 문서에서 자주 반복되느냐..
- 높을수록 중요하지 않음.
- 낮으면 특수 문서에서만 나타난다는 뜻
- IDF = 1/DF

> TF_IDF
- 특정 단어의 상대적인 빈도를 나타내는 값
- 값이 클수록 (DF가 작음)내 문서에만 많이 언급되는 단어 -> 다른 문서에서는 언급 안됨=> 고유성. 대표성이 큼
- 값이 작을수록(DF가 큼)  다른 문서에 잘 언급하는 단어 -> 현재 문서와 관련 없음.
- 값이 클수록 내 문장을 대표하는(특징적인) 단어라는 뜻. 
- 단어의 순서는 고려하지 않음.
- tf_IDF값이 높으려면 tf는 높고, idf는 높아야 함.-> 나만 많이 사용
- tf-idf값이 낮으려면 tf는 낮고, idf도 낮아야 함.-> 안쓰는 단어

### tf-idf 계산
- 문서 수만큼 존재함-> 행렬값으로 나타남.
- 단어로만 부과되는 값이 아니라, 내 문장내에서 부여됨.
- tf-idf = tf * log(n/df)

> df를 단순 수치로 비교할 경우
- 전체 100건 중 사과라는 단어가  90이 나왔으면 df = 90 -> 확률은 0.09
- 전체 10건중 사과가 5건 나왔으면 df = 5 -> 확률은 0.5
- 단순 수치로 비교하면 안되기 때문에 df값을 0~1로 나누기 위해서
- df를 n으로 나눠줌
- idf는 역수이므로 n/df
- df값은 문장이 많을 수록 기하급수적으로 커지기 때문에
- 전체 문장 수로 나눠주면 최대값이 1이 됨.


> log로 나누는 이유
- 전체 문서가 1000건중 2개 나왔으면 idf = 500
- 전체 문서  10만건 중 2개 나왔으면 idf = 5만
- tf값은 그리 클 수가 없는 값인데 idf가 너무 크기 때문에 스케일을 맞추기 위해서 log값을 취함.


In [1]:
sent = ["오늘 휴일", 
        "휴일 오늘", 
        "휴일 인 오늘 도 서쪽 을 중심 으로 폭염 이 이어졌는데요, 내일 은 반가운 비 소식 이 있습니다.", 
        "폭염 을 피해서 휴일 에 놀러왔다가 갑작스런 비 로 인해 망연자실 하고 있습니 다.", 
        " 내일 은 반가운 비 소식 이 있습니다."] 

In [82]:
words= ''.join(sent)
words =words.split()
print(words)

['오늘', '휴일휴일', '오늘휴일', '인', '오늘', '도', '서쪽', '을', '중심', '으로', '폭염', '이', '이어졌는데요,', '내일', '은', '반가운', '비', '소식', '이', '있습니다.폭염', '을', '피해서', '휴일', '에', '놀러왔다가', '갑작스런', '비', '로', '인해', '망연자실', '하고', '있습니', '다.', '내일', '은', '반가운', '비', '소식', '이', '있습니다.']


In [69]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(sent) #문장 벡터화 진행

In [70]:
print(tfidf_matrix.toarray().shape)

# 5 : 문서의 수 -> 문서마다 결정
# 18: 단어 수 

(5, 18)


In [71]:
print(tfidf_matrix)

  (0, 17)	0.6437444595062429
  (0, 7)	0.7652405313723362
  (1, 17)	0.6437444595062429
  (1, 7)	0.7652405313723362
  (2, 12)	0.28487998702172107
  (2, 6)	0.28487998702172107
  (2, 4)	0.28487998702172107
  (2, 1)	0.28487998702172107
  (2, 9)	0.35310140100264525
  (2, 14)	0.28487998702172107
  (2, 8)	0.35310140100264525
  (2, 13)	0.35310140100264525
  (2, 5)	0.35310140100264525
  (2, 17)	0.19893117008503197
  (2, 7)	0.23647612349029334
  (3, 11)	0.3542556015420614
  (3, 16)	0.3542556015420614
  (3, 3)	0.3542556015420614
  (3, 10)	0.3542556015420614
  (3, 0)	0.3542556015420614
  (3, 2)	0.3542556015420614
  (3, 15)	0.3542556015420614
  (3, 14)	0.28581118874948447
  (3, 17)	0.1995814265359179
  (4, 12)	0.5
  (4, 6)	0.5
  (4, 4)	0.5
  (4, 1)	0.5


In [72]:
print(type(tfidf_matrix))

<class 'scipy.sparse.csr.csr_matrix'>


In [87]:
tfidf_mat =tfidf_matrix.toarray()
tfidf_mat

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.76524053, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.64374446],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.76524053, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.64374446],
       [0.        , 0.28487999, 0.        , 0.        , 0.28487999,
        0.3531014 , 0.28487999, 0.23647612, 0.3531014 , 0.3531014 ,
        0.        , 0.        , 0.28487999, 0.3531014 , 0.28487999,
        0.        , 0.        , 0.19893117],
       [0.3542556 , 0.        , 0.3542556 , 0.3542556 , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.3542556 , 0.3542556 , 0.        , 0.        , 0.28581119,
        0.3542556 , 0.3542556 , 0.19958143],
    

In [83]:
# 단어 사전 확인
features = tfidf_vectorizer.get_feature_names()
print(len(features), features)


# 한 단어는 빼고,  겹치는 단어는 모두 제거되었음.

18 ['갑작스런', '내일', '놀러왔다가', '망연자실', '반가운', '서쪽', '소식', '오늘', '으로', '이어졌는데요', '인해', '있습니', '있습니다', '중심', '폭염', '피해서', '하고', '휴일']


In [84]:
print(set(words)- set(features))

{'을', '오늘휴일', '휴일휴일', '인', '에', '로', '은', '도', '다.', '있습니다.', '이', '비', '이어졌는데요,', '있습니다.폭염'}


> 결과 해석
- tf-idf의 값이 0이라는 것은 그 문장에 안나왔다는 뜻 (tf = 0일테니까)
- tf-idf의 값이 높다는 것은 내 문장에는 많이 나오고,다른 문장에서는 안나와야 함.
- 

#### 특정 단어의 tf-idf값 확인해보자

In [101]:
# 단어 사전에서의 인덱스 출력
tfidf_vectorizer.vocabulary_.get('오늘')

7

In [104]:
mat = np.asarray(tfidf_matrix.toarray())
srch = ['오늘', '휴일']
print([tfidf_vectorizer.vocabulary_.get(i) for i in srch ])
       

[7, 17]


In [109]:
srch_dtm = mat[:, [tfidf_vectorizer.vocabulary_.get(i) for i in srch]] # mat[:, [7,17]]
srch_dtm

array([[0.76524053, 0.64374446],
       [0.76524053, 0.64374446],
       [0.23647612, 0.19893117],
       [0.        , 0.19958143],
       [0.        , 0.        ]])


> 해석
- 값이 0인 것은 아예 그 문장에서 그 단어가 안나타났다는 뜻 (tf = 0이므로)
- 값이 0이 아니라는 것은 그 문장에 나타났다는 뜻
- 각 문장에 1번씩 나왔는데,왜 tf-idf값이 다른가? -> 값이 낮은 것은 tf값은 같지만, idf가 낮지 때문 -> 다른 문장에서 많이 사용되었다는 뜻

#### 문장과 문장의 유사도 비교1
- 문장끼리 유사도를 비교할 수도 있고,
- 분류도 가능(비슷한 문장끼지 classification을 해서)

In [110]:
np.sum(srch_dtm, axis = 1)



array([1.40898499, 1.40898499, 0.43540729, 0.19958143, 0.        ])

> 문장별로 더했을 때 값이 클수록
- '오늘', '휴일'이 각 문장들과 관련성이 높다는 의미

In [112]:
score = srch_dtm.sum(axis = 1)
print(score)

[1.40898499 1.40898499 0.43540729 0.19958143 0.        ]


In [113]:
for i in range(len(score)):
    if score[i] >0:
        print('{}/score:{}'.format(sent[i], score[i]))

오늘 휴일/score:1.408984990878579
휴일 오늘/score:1.408984990878579
휴일 인 오늘 도 서쪽 을 중심 으로 폭염 이 이어졌는데요, 내일 은 반가운 비 소식 이 있습니다./score:0.4354072935753253
폭염 을 피해서 휴일 에 놀러왔다가 갑작스런 비 로 인해 망연자실 하고 있습니 다./score:0.1995814265359179


#### 문장과 문장의 유사도 1

In [119]:
mat = np.asarray(tfidf_matrix.toarray())
srch=['갑작스런', '망연자실']
print([  tfidf_vectorizer.vocabulary_.get(i) for i in srch])

srch_dtm = mat[:, [  tfidf_vectorizer.vocabulary_.get(i) for i in srch]]

#srch_dtm = mat[:, [ 7,17]]   
#srch_dtm = mat[:, 7]   

print(srch_dtm)

[0, 3]
[[0.        0.       ]
 [0.        0.       ]
 [0.        0.       ]
 [0.3542556 0.3542556]
 [0.        0.       ]]


In [118]:
score = srch_dtm.sum(axis=1)
print(score)

for i in range(len(score)):
    if score[i] > 0:
        print('{} / score : {}'.format(sent[i], score[i]))  

[0.         0.         0.28487999 0.3542556  0.5       ]
휴일 인 오늘 도 서쪽 을 중심 으로 폭염 이 이어졌는데요, 내일 은 반가운 비 소식 이 있습니다. / score : 0.28487998702172107
폭염 을 피해서 휴일 에 놀러왔다가 갑작스런 비 로 인해 망연자실 하고 있습니 다. / score : 0.3542556015420614
 내일 은 반가운 비 소식 이 있습니다. / score : 0.5


In [125]:
mat = np.asarray(tfidf_matrix.toarray())
srch=['갑작스런', '휴일']
print([  tfidf_vectorizer.vocabulary_.get(i) for i in srch])

srch_dtm = mat[:, [  tfidf_vectorizer.vocabulary_.get(i) for i in srch]]

#srch_dtm = mat[:, [ 7,17]]   
#srch_dtm = mat[:, 7]   

print(srch_dtm)

[0, 17]
[[0.         0.64374446]
 [0.         0.64374446]
 [0.         0.19893117]
 [0.3542556  0.19958143]
 [0.         0.        ]]


In [126]:
score = srch_dtm.sum(axis=1)
print(score)

for i in range(len(score)):
    if score[i] > 0:
        print('{} / score : {}'.format(sent[i], score[i]))  

[0.64374446 0.64374446 0.19893117 0.55383703 0.        ]
오늘 휴일 / score : 0.6437444595062429
휴일 오늘 / score : 0.6437444595062429
휴일 인 오늘 도 서쪽 을 중심 으로 폭염 이 이어졌는데요, 내일 은 반가운 비 소식 이 있습니다. / score : 0.19893117008503197
폭염 을 피해서 휴일 에 놀러왔다가 갑작스런 비 로 인해 망연자실 하고 있습니 다. / score : 0.5538370280779793


#### 유사도 모델 만들기: SGDClassifier()이용

- train data

In [128]:
sentences = ['This is the first document.',
              'This is the second document.',
              'And the third one.',
              'Is this the first document?']
vect = TfidfVectorizer()
X = vect.fit_transform(sentences)
y = [1,2,3,4]

In [133]:
# 모델
model =  SGDClassifier(loss='perceptron')  # 단층 퍼셉트론 모델 이용하기

model.fit(X, y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='perceptron',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [134]:
X_pred = vect.transform(['My new document third'])
y_pred = model.predict(X_pred)
print(y_pred)

[3]


In [138]:
vect.vocabulary_

{'this': 8,
 'is': 3,
 'the': 6,
 'first': 2,
 'document': 1,
 'second': 5,
 'and': 0,
 'third': 7,
 'one': 4}

> test과정에서
- train에서 만들어진 단어 사전에 있는 단어만 고려하여 파악하기
- 실제로 document, third로만 tfidf값을 계산해서 유사도 비교

#### 분류 모델 만들기

- train data

In [135]:
sentences = ['This is the first document.',
              'This is the second document.',
              'And the third one.',
              'Is this the first document?']
vect = TfidfVectorizer()
X = vect.fit_transform(sentences)
y = [1,1,2,2]  # y라벨을 classification할수 있게 주기

In [136]:
# 모델
model =  SGDClassifier(loss='perceptron')  # 단층 퍼셉트론 모델 이용하기

model.fit(X, y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='perceptron',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [137]:
X_pred = vect.transform(['My new document third'])
y_pred = model.predict(X_pred)
print(y_pred)

[2]
